## Imports

In [ ]:
import numpy as np
import subprocess
import os
import pandas as pd
import matplotlib.pyplot as plt
import random
import array
import csv

# import deap packages required
from deap import algorithms
from deap import base
from deap import creator
from deap import tools
import pandas as pd

## Load test instance 

In [ ]:
myinst = "./artGalleryTestInstances/rooms.csv"
instance_file = myinst
instance_size  = 100
num_cells = instance_size * instance_size # total number of cells in the grid

# create a grid that specifies the walls that can be used later to check that no cameras are positioned on walls
walls = np.zeros(instance_size * instance_size)

with open(myinst) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter = ',')
    for line in csv_reader:
        column = int(line[0])
        row = int(line[1])
        oneD_index = (row * instance_size) + column
        walls[oneD_index] = 1

## Executables wrappers

In [ ]:
path_binary = "ECO-Coursework-Executables/bit_cam_napier.exe"
path_binary_vis = "ECO-Coursework-Executables/bit_cam_napier_visualisation.exe"

# Do NOT modify this code - this calls an external binary with a solution
def objective_function(x, instance_size, nb_cameras, instance_file):
    params = ['%.16g' % a for a in x]
    cmd = [path_binary,str(instance_size),str(nb_cameras)]+params+[instance_file]
    s = subprocess.check_output(cmd)
    return float(s)


# Do NOT modify: this checks whether a camera is positioned on top of wall in a solution
def check_walls(solution, inst):
    clashes=0
    for i in range(0, len(solution)):
        if (walls[i] == 1 and solution[i]==1):
            clashes+=1
            
    return(clashes)

## Evaluation function

In [ ]:
# this is the eval function called from DEAP: you can modify this to adapt the fitness for invalid solutions. The fitness of a valid solution
# is obtained by calling the binary executable

# This is the function where you will likely do most work!

def eval_function(individual):

    solution = [] # list of length equivalent to number of cells in the grid, where each value is 0 or 1

    # convert individual to the solution list - this depends on the representation
    for i in range(0, len(individual)):
        solution.append(individual[i])

    # count cameras in solution
    total_cameras = np.sum(solution)
    
    # check if cameras on walls
    if instance_file == "":
        cameras_on_walls = 0
    else:
        cameras_on_walls = check_walls(solution, instance_file) 

    print(total_cameras)
    
    invalid_penalty=20000
    not_covered_penalty_factor = 11000
                                        
    # assign fitness after checking for validity
    if  total_cameras < 1:
        fitness =  invalid_penalty # no cameras
    

    else:
        # only call this if the solution is not invalid
        coverage = objective_function(solution, instance_size, total_cameras, instance_file)
    
        if coverage < 1.0:
            # decide how to penalise this solution which does not provide 100% coverage 
            fitness = not_covered_penalty_factor  # you should modify this
        else:
            fitness = total_cameras  # fitness is the number of cameras used (minimise)

    return fitness,

## Setup the EA

In [ ]:
# SETUP THE EA
# define the fitness class and creare an individual class
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)


# create a toolbox
toolbox = base.Toolbox()

# Attribute generator
toolbox.register("attr_bool", random.randint, 0, 1)

toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, num_cells)

toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# register all operators we need with the toolbox
toolbox.register("evaluate", eval_function)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=2)

## Main EA definition

In [ ]:
def main():
    
    # choose a population size: e.g. 200
    pop = toolbox.population(n=200)
    
    # keep track of the single best solution found
    hof = tools.HallOfFame(1)
 
    # create a statistics object: we can log what ever statistics we want using this. We use the numpy Python library
    # to calculate the stats and label them with convenient labels
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)
    
    # run the algorithm: we need to tell it what parameters to use
    # cxpb = crossover probability; mutpb = mutation probability; ngen = number of iterations
    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.6, mutpb=0.01, ngen=400, stats=stats, halloffame=hof, verbose=True)
    
    return pop, log, hof

## Run the EA

In [ ]:

pop, log, hof = main()

best = hof[0].fitness.values[0]   # best fitness found is stored at index 0 in the hof list

# look in the logbook to see what generation this was found at
max = log.select("max")  # max fitness per generation stored in log

for i in range(400):  # set to ngen
        fit = max[i]
        if fit == best:
            break        
        
print("min fitness found is %s at generation %s" % (best, i))